## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-10-16-52-39 +0000,nyt,White House Signals Federal Layoffs Have Begun...,https://www.nytimes.com/2025/10/10/us/politics...
1,2025-10-10-16-51-36 +0000,nyt,"Under Trump, E.P.A. Explored if Abortion Pills...",https://www.nytimes.com/2025/10/10/us/politics...
2,2025-10-10-16-51-28 +0000,nyt,Deaths Are Reported in Explosion at Tennessee ...,https://www.nytimes.com/2025/10/10/us/explosio...
3,2025-10-10-16-46-57 +0000,nyt,Live Updates: María Corina Machado of Venezuel...,https://www.nytimes.com/live/2025/10/10/world/...
4,2025-10-10-16-46-57 +0000,bbc,Melania Trump says Putin responded to her plea...,https://www.bbc.com/news/articles/cj075gq5n81o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2321/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,55
150,gaza,37
152,deal,26
28,peace,26
27,nobel,23


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
347,2025-10-09-19-02-29 +0000,nypost,Rep. Andy Barr latest to nominate Trump for No...,https://nypost.com/2025/10/09/us-news/rep-andy...,200
167,2025-10-10-09-06-09 +0000,nypost,Why Trump wasn’t awarded Nobel Peace Prize for...,https://nypost.com/2025/10/10/us-news/why-trum...,199
81,2025-10-10-14-32-26 +0000,nypost,"Fresh off Nobel Peace Prize win, opposition le...",https://nypost.com/2025/10/10/world-news/nobel...,188
306,2025-10-09-20-54-59 +0000,nypost,Jubilant Trump vows Gaza peace deal will ‘move...,https://nypost.com/2025/10/09/us-news/jubilant...,179
279,2025-10-09-22-13-20 +0000,nypost,Obama called out for omitting Trump in post ap...,https://nypost.com/2025/10/09/us-news/obama-ap...,163


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
347,200,2025-10-09-19-02-29 +0000,nypost,Rep. Andy Barr latest to nominate Trump for No...,https://nypost.com/2025/10/09/us-news/rep-andy...
124,80,2025-10-10-11-33-02 +0000,nyt,"How Letitia James, Now Indicted, Will Stay as ...",https://www.nytimes.com/2025/10/10/us/politics...
81,62,2025-10-10-14-32-26 +0000,nypost,"Fresh off Nobel Peace Prize win, opposition le...",https://nypost.com/2025/10/10/world-news/nobel...
77,48,2025-10-10-14-45-15 +0000,latimes,"Sister Jean, nun who became a national icon du...",https://www.latimes.com/sports/story/2025-10-1...
214,48,2025-10-10-03-22-00 +0000,wsj,"As tensions rise on Capitol Hill, Republican a...",https://www.wsj.com/politics/policy/government...
167,47,2025-10-10-09-06-09 +0000,nypost,Why Trump wasn’t awarded Nobel Peace Prize for...,https://nypost.com/2025/10/10/us-news/why-trum...
72,44,2025-10-10-15-00-00 +0000,wsj,"From cabinet secretaries to state legislators,...",https://www.wsj.com/politics/the-beauty-queens...
233,38,2025-10-10-01-00-00 +0000,wsj,China’s Rare-Earth Escalation Threatens Trade ...,https://www.wsj.com/world/china/china-trade-ra...
250,37,2025-10-09-23-54-00 +0000,wsj,The Bureau of Labor Statistics will bring some...,https://www.wsj.com/politics/policy/federal-go...
36,35,2025-10-10-16-02-28 +0000,wapo,"Peru’s president removed amid rising crime, co...",https://www.washingtonpost.com/world/2025/10/1...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
